<a href="https://colab.research.google.com/github/Madhu-712/Gemini_Langchainagents/blob/main/Gemini_Flash_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini API: Customer Support Agent

### Install Packages

In [1]:
def is_prime(n):
  """Returns True if n is a prime number, False otherwise."""
  if n <= 1:
    return False
  for i in range(2, int(n ** 0.5) + 1):
    if n % i == 0:
      return False
  return True

primes = []
num = 2
while len(primes) < 200:
  if is_prime(num):
    primes.append(num)
  num += 1

sum_of_primes = sum(primes)
print(f'The sum of the first 200 prime numbers is: {sum_of_primes}')

The sum of the first 200 prime numbers is: 111587


In [2]:
!pip install -U -q google-generativeai

### Import packages

Import the necessary packages.

In [3]:
import pathlib
import textwrap
import time

import google.generativeai as genai
import google.ai.generativelanguage as glm

from IPython import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Set up your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In [4]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

### Customer Support Chatbot

In [5]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

### Create Gemini Client

In [6]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return] # list of all available tools
)

In [7]:
model._tools.to_proto()

[function_declarations {
   name: "get_order_status"
   description: "Fetches the status of a given order ID."
   parameters {
     type_: OBJECT
     properties {
       key: "order_id"
       value {
         type_: STRING
       }
     }
     required: "order_id"
   }
 }
 function_declarations {
   name: "initiate_return"
   description: "Initiates a return for a given order ID with a specified reason."
   parameters {
     type_: OBJECT
     properties {
       key: "reason"
       value {
         type_: STRING
       }
     }
     properties {
       key: "order_id"
       value {
         type_: STRING
       }
     }
     required: "order_id"
     required: "reason"
   }
 }]

### alway use the model in chat mode for function calling

In [8]:
chat = model.start_chat(enable_automatic_function_calling=True)

While this was all handled automatically, if you need more control, you can:

- Leave the default `enable_automatic_function_calling=False` and process the `glm.FunctionCall` responses yourself.
- Or use `GenerativeModel.generate_content`, where you also need to manage the chat history.

In [9]:
response = chat.send_message('What is the status of order 12345?')
response.text

'The order has shipped.\n'

In [10]:
response.candidates

[content {
  parts {
    text: "The order has shipped.\n"
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.036300492783387504
]

Examine the chat history to see the flow of the conversation and how function calls are integrated within it.

The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [11]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order has shipped.\n'}
--------------------------------------------------------------------------------


In [12]:
response = chat.send_message('I want to return order 11223 because it is defective.')
print(response.text)

OK. I have initiated a return for order 11223 because it is defective.



In [13]:
response.candidates

[content {
  parts {
    text: "OK. I have initiated a return for order 11223 because it is defective.\n"
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.049612754867190405
]

In [14]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order has shipped.\n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '11223', 'reason': 'defective'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'initia

### Sequential Function Calls or Nested Calls

Output of the first function call becomes the input to the second!

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [15]:
response = chat.send_message('Can you check the status of order 11223? If its delivered, please initiat return as it was the wrong order')
print(response.text)

Return initiated for order 11223 because it was the wrong order.



In [16]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order has shipped.\n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '11223', 'reason': 'defective'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'initia

# Sequential function calls - 2

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [17]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order')
print(response.text)

The order 67890 is currently processing.  I cannot initiate a return until it is delivered.



In [18]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.')
print(response.text)

Order 67890 is currently processing.  I cannot initiate a return or cancellation at this stage.



In [19]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order has shipped.\n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '11223', 'reason': 'defective'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'initia

### Adding a funciton to handle cancel

In [20]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {order_statuses[order_id]}."
    else:
        return "Order ID not found. Cannot cancel order."

add cancel to the list of functions.

In [21]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return, cancel_order]
)

In [22]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [23]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.')
print(response.text)

The order with ID 67890 is currently being processed.  I cannot initiate a return or cancellation at this time.



In [24]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order with ID 67890 is currently being processed.  I cannot initiate a return or cancellation at this time.\n'}
--------------------------------------------------------------------------------


### Parallel Function Calling. Can it handle those?

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [25]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return, cancel_order]
)

chat = model.start_chat(enable_automatic_function_calling=True)

In [26]:
response = chat.send_message("What is the status of order 12345? Also, can you cancel order 67890 and initiate a return for order 11223 because it is defective?")

In [27]:
response.text

'The status of order 12345 is Shipped. Order 67890 has been canceled successfully. A return has been initiated for order 11223 because it is defective.\n'

### How many functions it can handle

In [56]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {order_statuses[order_id]}."
    else:
        return "Order ID not found. Cannot cancel order."

def update_shipping_address(order_id: str, new_address: str) -> str:
    """Updates the shipping address for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Shipping address for order {order_id} has been updated to: {new_address}."
    else:
        return "Order ID not found. Cannot update shipping address."

def track_shipment(tracking_number: str) -> str:
    """Tracks the shipment with the given tracking number."""
    # Mock data for example purposes
    tracking_info = {
        "TRACK123": "In Transit",
        "TRACK456": "Delivered",
        "TRACK789": "Out for Delivery"
    }
    return tracking_info.get(tracking_number, "Tracking number not found.")

def apply_discount(order_id: str, discount_code: str) -> str:
    """Applies a discount to the given order ID."""
    # Mock data for example purposes
    valid_discount_codes = ["DISCOUNT10", "SAVE20"]
    if order_id in ["12345", "67890", "11223"]:
        if discount_code in valid_discount_codes:
            return f"Discount code {discount_code} applied to order {order_id}."
        else:
            return f"Invalid discount code: {discount_code}."
    else:
        return "Order ID not found. Cannot apply discount."

def change_payment_method(order_id: str, payment_method: str) -> str:
    """Changes the payment method for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Payment method for order {order_id} has been changed to: {payment_method}."
    else:
        return "Order ID not found. Cannot change payment method."

def provide_invoice(order_id: str) -> str:
    """Provides an invoice for the given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Invoice for order {order_id} has been sent to your email."
    else:
        return "Order ID not found. Cannot provide invoice."

def extend_warranty(order_id: str, years: int) -> str:
    """Extends the warranty for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Warranty for order {order_id} has been extended by {years} years."
    else:
        return "Order ID not found. Cannot extend warranty."

def check_product_availability(product_id: str) -> str:
    """Checks the availability of a product with the given product ID."""
    # Mock data for example purposes
    product_availability = {
        "PROD123": "In Stock",
        "PROD456": "Out of Stock",
        "PROD789": "Limited Stock"
    }
    return product_availability.get(product_id, "Product ID not found.")

In [57]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[
        get_order_status, initiate_return, cancel_order, update_shipping_address,
        track_shipment, apply_discount, change_payment_method, provide_invoice,
        extend_warranty, check_product_availability
    ]
)

In [58]:
chat = model.start_chat()

In [59]:
response = chat.send_message("Can you change the payment method to'COD'and provide the Invoice for 12345 order id")

In [47]:
response.candidates

[content {
  parts {
    function_call {
      name: "change_payment_method"
      args {
        fields {
          key: "payment_method"
          value {
            string_value: "COD"
          }
        }
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  parts {
    text: "\n"
  }
  parts {
    function_call {
      name: "provide_invoice"
      args {
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.00821373072163812
]

In [48]:
# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

change_payment_method(payment_method=COD, order_id=12345)
provide_invoice(order_id=12345)


In [51]:
responses = {
    'change_payment_method': change_payment_method(payment_method="COD",order_id="12345"),
    'provide_invoice': provide_invoice(order_id="12345")
}

In [52]:
change_payment_method(payment_method="COD",order_id="12345")

'Payment method for order 12345 has been changed to: COD.'

In [53]:
# Build the response parts.
response_parts = [
    glm.Part(function_response=glm.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

responses = chat.send_message(response_parts)
print(responses.text)

The payment method for order 12345 has been changed to COD. The invoice has been sent to your email.



In [31]:
response = chat.send_message("What is the status of order 12345? Can you update the address to 123 Main St, Anytown USA?")
# response.text

In [32]:
response.candidates

[content {
  parts {
    function_call {
      name: "get_order_status"
      args {
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "update_shipping_address"
      args {
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
        fields {
          key: "new_address"
          value {
            string_value: "123 Main St, Anytown USA"
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -1.1073951961742223e-06
]

In [33]:
# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

get_order_status(order_id=12345)
update_shipping_address(new_address=123 Main St, Anytown USA, order_id=12345)


In [34]:
responses = {
    'get_order_status': get_order_status(order_id="12345"),
    'update_shipping_address': update_shipping_address(order_id="12345", new_address="123 Main St, Anytown USA")
}

In [35]:
get_order_status(order_id="12345")

'Shipped'

In [36]:
# Build the response parts.
response_parts = [
    glm.Part(function_response=glm.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

responses = chat.send_message(response_parts)
print(responses.text)

The order status is shipped. The shipping address has been updated to 123 Main St, Anytown, USA.

